In [1]:
# INÍCIO - SISTEMA AGENTE HELP CURRÍCULOS ################################################
%pip -q install google-genai

In [2]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata, output

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
output.enable_custom_widget_manager()

In [ ]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk
!pip install -q ipycanvas

In [4]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
from ipycanvas import Canvas
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [5]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [6]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  #text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [7]:
##########################################
# --- Agente 1: Cadastro de Currículo --- #
##########################################

def agente_cadastro_curriculo(info_nome,info_idade,info_endereco,info_email,
info_telefone,info_profissao,info_formacao,info_cursos,
info_experiencia_empresa1,info_experiencia_funcao1,info_experiencia_tempo1,
info_experiencia_empresa2,info_experiencia_funcao2,info_experiencia_tempo2,
info_experiencia_empresa3,info_experiencia_funcao3,info_experiencia_tempo3,
info_experiencia_empresa4,info_experiencia_funcao4, info_experiencia_tempo4,
info_experiencia_empresa5,info_experiencia_funcao5,info_experiencia_tempo5,
                              data_de_hoje):

    cadastro_curriculo = Agent(
        name="agente_cadastro_curriculo",
        model="gemini-2.0-flash",
        instruction=f"""
        Você é um assistente de pesquisa e cadastro de currículos.
        A sua tarefa é pegar as informações abaixo:
        Nome: {info_nome}, Endereço: {info_endereco}, email:{info_email},
        Telefone: {info_telefone}, Profissão: {info_profissao}, Formação:{info_formacao},
        Cursos:{info_cursos},
        Experiência Profissional 1 na empresa: {info_experiencia_empresa1}
        com a função de: {info_experiencia_funcao1} e no perído de: {info_experiencia_tempo1}

        Experiência Profissional 2 na empresa: {info_experiencia_empresa2}
        com a função de: {info_experiencia_funcao2} e no perído de: {info_experiencia_tempo2}

        Experiência Profissional 3 na empresa: {info_experiencia_empresa3}
        com a função de: {info_experiencia_funcao3} e no perído de: {info_experiencia_tempo3}

        Experiência Profissional 4 na empresa: {info_experiencia_empresa4}
        com a função de: {info_experiencia_funcao4} e no perído de: {info_experiencia_tempo4}

        Experiência Profissional 5 na empresa: {info_experiencia_empresa5}
        com a função de: {info_experiencia_funcao5} e no perído de: {info_experiencia_tempo5}

        E agora crie um currículo profissional e bem formal com base nesses dados.
        Siga a ordem abaixo:
        Nome será como cabeçalho e terá a Fonte bem grande, e um alinhamento centralizado.
        Dados pessoais terão alinhamento centralizado.
        Profissão terão alinhamento centralizado.
        Resumo
        Formação ou escolaridade
        Cursos
        Experiência Profissional

        Caso exista alguma informação não incluída, você pode incluir.

        """,
        description="Agente gera um currículo padrão baseado nas informações passadas pelo usuário",
        tools=[google_search]
    )

    entrada_do_cadastro_curriculo = f"Tópico: {info_nome} {info_profissao}\nData de hoje: {data_de_hoje}"
    cadastro = call_agent(cadastro_curriculo, entrada_do_cadastro_curriculo)

    return cadastro

In [8]:
################################################
# --- Agente 2: criar arquivo docx --- #
################################################
def agente_criar_docx(info_nome,info_idade,info_endereco,info_email,
info_telefone,info_profissao,info_formacao,info_cursos,
info_experiencia_empresa1,info_experiencia_funcao1,info_experiencia_tempo1,
info_experiencia_empresa2,info_experiencia_funcao2,info_experiencia_tempo2,
info_experiencia_empresa3,info_experiencia_funcao3,info_experiencia_tempo3,
info_experiencia_empresa4,info_experiencia_funcao4, info_experiencia_tempo4,
info_experiencia_empresa5,info_experiencia_funcao5,info_experiencia_tempo5,
curriculo_cadastrado, data_de_hoje):

    # Inserir as instruções do Agente Planejador #################################################

    with open('texto-canvas.docx', 'w')as arquivo:
      arquivo.write(curriculo_cadastrado)
    print("Arquivo texto.canvas.docx criado");

    docxcriador = Agent(
        name="agente_criar_docx",
        model="gemini-2.0-flash",
        instruction=f"""
        """,
        description="Agente que cria docx",
    )
    entrada_criar_docx = f"Currículo do {info_nome} {info_profissao} \nDOCX criado: {curriculo_cadastrado}"
    # Executa o agente
    criar_docx = call_agent(docxcriador, entrada_criar_docx)
    return criar_docx

In [9]:
#####################################################################
# --- Agente 3: Buscador de Vagas com impressão de arquivo DOCX --- #
#####################################################################
def agente_buscador_de_vagas(info_nome,info_endereco,info_profissao,
      curriculo_cadastrado, data_de_hoje):

    buscador_de_vagas = Agent(
        name="agente_buscador_de_vagas",
        model="gemini-2.0-flash",
        instruction=f"""
        Você é um especialista em pesquisa de vagas de emprego.
        A sua tarefa é:
        1 - Com a ferramenta(google_search) buscar vagas de emprego abertas nas empresas no periodo próximo da data de {data_de_hoje},
        na profissão de {info_profissao} e com preferência no estado do endereço{info_endereco}
        que mais se aproximam das qualidades informadas no currículo {curriculo_cadastrado}.

        2 - Liste todas as vagas ordenadas pela vaga aberta na data mais recente, com nome da empresa, url do site, link de inscrição.

        Caso exista alguma informação não incluída, você pode incluir.

        """,
        description="Agente lista informação de vagas abertas",
        tools=[google_search]
    )

    entrada_do_buscador_de_vagas = f"Buscador de vagas para: {info_nome} {info_profissao}\nBuscador de vagas: {agente_buscador_de_vagas}"
    # Executa o agente
    buscador = call_agent(buscador_de_vagas, entrada_do_buscador_de_vagas)

    with open('buscador_de_vagas.docx', 'w')as arquivo:
      arquivo.write(buscador)
    print("""
    \n--- 📝 Criado Arquivo .DOCX com Lista de Vagas ---\n

            buscador_de_vagas.docx criado

      ----------------------------------------------------
            """);

    return buscador

In [ ]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print(""" 🚀🚀🚀 Iniciando o Sistema de Criação de Currículos com 3 Agentes 🚀🚀🚀

    🚀Obs: Caso o campo não seja (Obrigatório), click [ENTER] para seguir nos próximos campos!!! 🚀
      """)
# --- Obter Informações do Usuário ---

info_nome = input("❓(Obrigatório) Por favor, digite o seu Nome Completo: ")
info_idade = input("❓ (Obrigatório) Por favor, digite a sua idade (Somente Números): ")
info_endereco = input("❓(Obrigatório) Por favor, digite o seu endereço: ")
info_email = input("❓ Por favor, digite o seu email: ")
info_telefone = input("❓ Por favor, digite o seu telefone com DDD: ")
info_profissao = input("❓(Obrigatório) Por favor, digite a profissão que atua ou deseja atuar: ")
info_formacao = input("❓ (Obrigatório) Por favor, digite a sua formação ou nível de escolaridade: ")
info_cursos = input("❓ Por favor, informe seus cursos: ")
info_experiencia_empresa1 = input("❓Experiência 1 : Por favor, digite a o nome da última empresa ou atual: ")
info_experiencia_funcao1 = input("❓ Experiência 1 : Por favor, digite o nome da função: ")
info_experiencia_tempo1 = input("❓ Experiência 1 : Por favor, digite o tempo na função: ")
info_experiencia_empresa2 = input("❓Experiência 2: Por favor, digite o nome da última empresa ou atual: ")
info_experiencia_funcao2 = input("❓ Experiência 2: Por favor, digite o nome da função: ")
info_experiencia_tempo2 = input("❓ Experiência 2: Por favor, digite o tempo na função: ")
info_experiencia_empresa3 = input("❓Experiência 3 : Por favor, digite o nome última empresa ou atual: ")
info_experiencia_funcao3 = input("❓ Experiência 3: Por favor, digite o nome da função: ")
info_experiencia_tempo3 = input("❓ Experiência 3: Por favor, digite o tempo na função: ")
info_experiencia_empresa4 = input("❓Experiência 4: Por favor, digite o nome da última empresa ou atual: ")
info_experiencia_funcao4 = input("❓ Experiência 4: Por favor, digite o nome da função: ")
info_experiencia_tempo4 = input("❓ Experiência 4: Por favor, digite o tempo na função: ")
info_experiencia_empresa5 = input("❓Experiência 5: Por favor, digite o nome da empresa anterior: ")
info_experiencia_funcao5 = input("❓ Experiência 5: Por favor, digite o nome da função: ")
info_experiencia_tempo5 = input("❓ Experiência 5: Por favor, digite o tempo na função: ")

# Lógica do sistema de agentes ################################################
if not info_nome:
    print(" OPs!  Você esqueceu de digitar o seu nome!")
if not info_idade:
    print(" OPs!  Você esqueceu de digitar o sua idade!")
if not info_endereco:
    print(" OPs!  Você esqueceu de digitar o seu endereço!")
if not info_profissao:
    print(" OPs!  Você esqueceu de digitar a sua profissão ou cargo que deseja atuar!")
if not info_formacao:
    print(" OPs!  Você esqueceu de digitar a formação ou escolaridade!")
else:
    print(f"Maravilha! Vamos então criar um currículo sobre o {info_nome} na profissão de {info_profissao}")

    # Agente para Cadastro de Currículo ################################################
    curriculo_cadastrado = agente_cadastro_curriculo(info_nome,info_idade,info_endereco,info_email,
      info_telefone,info_profissao,info_formacao,info_cursos,
      info_experiencia_empresa1,info_experiencia_funcao1,info_experiencia_tempo1,
      info_experiencia_empresa2,info_experiencia_funcao2,info_experiencia_tempo2,
      info_experiencia_empresa3,info_experiencia_funcao3,info_experiencia_tempo3,
      info_experiencia_empresa4,info_experiencia_funcao4, info_experiencia_tempo4,
      info_experiencia_empresa5,info_experiencia_funcao5,info_experiencia_tempo5,
                                                     data_de_hoje)
    print("\n--- 📝 Resultado do cadastro curricular 1 (cadastro) ---\n")
    display(to_markdown(curriculo_cadastrado))
    print("--------------------------------------------------------------")

    print("\n--- 📝 Resultado do criação do docx 2 (docx) ---\n")

    # Agente para Criação de Arquivo DOCX ################################################
    criar_docx = agente_criar_docx(info_nome,info_idade,info_endereco,info_email,
      info_telefone,info_profissao,info_formacao,info_cursos,
      info_experiencia_empresa1,info_experiencia_funcao1,info_experiencia_tempo1,
      info_experiencia_empresa2,info_experiencia_funcao2,info_experiencia_tempo2,
      info_experiencia_empresa3,info_experiencia_funcao3,info_experiencia_tempo3,
      info_experiencia_empresa4,info_experiencia_funcao4, info_experiencia_tempo4,
      info_experiencia_empresa5,info_experiencia_funcao5,info_experiencia_tempo5,
      curriculo_cadastrado, data_de_hoje)

    #display(to_markdown(criar_docx))
    print("--------------------------------------------------------------")

    # Agente para buscar vagas e emitir DOCX da lista ################################################
    buscar_vagas = agente_buscador_de_vagas(info_nome,info_endereco,info_profissao,
      curriculo_cadastrado, data_de_hoje)
    print("\n--- 📝 Resultado do buscador de vagas 3 (docx) ---\n")
    display(to_markdown(buscar_vagas))
    print("--------------------------------------------------------------")

    # FIM - SISTEMA AGENTE HELP CURRÍCULOS ################################################